In [1]:
import pandas as pd
import numpy as np
import talib
from talib import abstract
from talib import stream
from talib.abstract import *
import glob
from datetime import timedelta

import warnings
warnings.filterwarnings('ignore')

### Prepro utils

In [2]:
def date_split_fun(data):
    date_columns = data["Date"].astype(str).str.split(" ", expand = True)
    data["Date"] = date_columns.iloc[:, 0]
    data["Time"] = date_columns.iloc[:, 1]
    data["Date"] = pd.to_datetime(data["Date"])
    return data

In [3]:
def label_fun(data, delta):
    return data[ ((data["Close"]/ data["Close"].shift(1)) > 1 ) & (((data["Low"] / data["Close"].shift(1))-1)*100  >= -delta)  ].index

In [4]:
def label_with_fee_fun(data, delta):
    return data[ ((data["Close_with_fee"]/ data["Close_with_fee"].shift(1)) > 1 ) & (((data["Low"]/ data["Close_with_fee"].shift(1))-1)*100  >= -delta)  ].index

In [7]:
def prepro_stock(dataset):
    df = dataset[['Date', "Time", 'Open', 'High', 'Low', 'Close', 'Volume']]
    df["z_open"]      = (df["Open"]  /df["Close"].shift(1))-1
    df["z_high"]      = (df["High"]  /df["Close"].shift(1))-1
    df["z_low" ]      = (df["Low"]   /df["Close"].shift(1))-1
    df["z_close"]     = (df["Close"] /df["Close"].shift(1))-1
    
    nan_5_data  = [np.nan] * 5
    nan_10_data = [np.nan] * 10
    nan_15_data = [np.nan] * 15
    nan_20_data = [np.nan] * 20
    nan_25_data = [np.nan] * 25
    nan_30_data = [np.nan] * 30

    df["z_d5"]  = nan_5_data  + ([((sum(df["Close"][i: ( 5 + (i+1)-1)]) / ( 5  *  df["Close"][ 5 +i: 6 +i])) - 1).values[0] for i in range(0, df.shape[0] -  5 )]) 
    df["z_d10"] = nan_10_data + ([((sum(df["Close"][i: (10 + (i+1)-1)]) / (10  *  df["Close"][10 +i:11 +i])) - 1).values[0] for i in range(0, df.shape[0] - 10 )]) 
    df["z_d15"] = nan_15_data + ([((sum(df["Close"][i: (15 + (i+1)-1)]) / (15  *  df["Close"][15 +i:16 +i])) - 1).values[0] for i in range(0, df.shape[0] - 15 )]) 
    df["z_d20"] = nan_20_data + ([((sum(df["Close"][i: (20 + (i+1)-1)]) / (20  *  df["Close"][20 +i:21 +i])) - 1).values[0] for i in range(0, df.shape[0] - 20 )]) 
    df["z_d25"] = nan_25_data + ([((sum(df["Close"][i: (25 + (i+1)-1)]) / (25  *  df["Close"][25 +i:26 +i])) - 1).values[0] for i in range(0, df.shape[0] - 25 )]) 
    df["z_d30"] = nan_30_data + ([((sum(df["Close"][i: (30 + (i+1)-1)]) / (30  *  df["Close"][30 +i:31 +i])) - 1).values[0] for i in range(0, df.shape[0] - 30 )]) 

    nan_5_data  = []
    nan_10_data = []
    nan_15_data = []
    nan_20_data = []
    nan_25_data = []
    nan_30_data = []
    

    # indicator
    MACD = abstract.Function('macd')
    Volume_MACD = abstract.Function('macd')
    RSI = abstract.Function('rsi')
    NATR = abstract.Function('natr')

    _,_, MACD = MACD(df["Close"], fastperiod=12, slowperiod=26, signalperiod=9 )
    _,_, V_MACD = Volume_MACD(df["Volume"], fastperiod=12, slowperiod=26, signalperiod=9 )
    RSI = RSI(df["Close"], timeperiod = 14 )
    NATR = NATR(df["High"], df["Low"], df["Close"], timeperiod=14)
    
    df["rsi"]         = RSI
    df["macd"]        = MACD
    df["v_macd"]      = V_MACD
    df["natr"]        = NATR
    
    df["rsi"]         = df["rsi"]
    df["macd"]        = df["macd"]
    df["v_macd"]      = df["v_macd"]
    df["natr"]        = df["natr"]
    
    # Trading Fee
    df["Close_with_fee"] = df["Close"] * 1.001
    
    
    # stop-loss trading strategy
    df["Target_Label"]     = 0
    df["Target_2.0_Label"] = 0
    df["Target_2.1_Label"] = 0
    df["Target_2.2_Label"] = 0
    df["Target_2.3_Label"] = 0

    df["Target_2.4_Label"] = 0
    df["Target_2.5_Label"] = 0
    df["Target_2.6_Label"] = 0
    df["Target_2.7_Label"] = 0
    df["Target_2.8_Label"] = 0
    df["Target_2.9_Label"] = 0
    df["Target_3.0_Label"] = 0
    
    df.loc[label_fun(df, 100.0), "Target_Label"]   = 1 
    df.loc[label_fun(df, 2.0 / 3), "Target_2.0_Label"] = 1 
    df.loc[label_fun(df, 2.2 / 3), "Target_2.2_Label"] = 1 
    df.loc[label_fun(df, 2.4 / 3), "Target_2.4_Label"] = 1 
    df.loc[label_fun(df, 2.6 / 3), "Target_2.6_Label"] = 1 
    df.loc[label_fun(df, 2.8 / 3), "Target_2.8_Label"] = 1 
    df.loc[label_fun(df, 3.0 / 3), "Target_3.0_Label"] = 1 
    df.loc[label_fun(df, 3.2 / 3), "Target_3.2_Label"] = 1 
    df.loc[label_fun(df, 3.4 / 3), "Target_3.4_Label"] = 1 
    df.loc[label_fun(df, 3.6 / 3), "Target_3.6_Label"] = 1 
    df.loc[label_fun(df, 3.8 / 3), "Target_3.8_Label"] = 1 
    df.loc[label_fun(df, 4.0 / 3), "Target_4.0_Label"] = 1 
#
    ## Stop-loss trading strategy with trading fee
    df["Target_Label_fee"]     = 0
    
    df["Target_2.0_Label_fee"] = 0
    df["Target_2.2_Label_fee"] = 0
    df["Target_2.4_Label_fee"] = 0
    df["Target_2.6_Label_fee"] = 0
    df["Target_2.8_Label_fee"] = 0
    df["Target_3.0_Label_fee"] = 0
    df["Target_3.2_Label_fee"] = 0
    df["Target_3.4_Label_fee"] = 0
    df["Target_3.6_Label_fee"] = 0
    df["Target_3.8_Label_fee"] = 0
    df["Target_4.0_Label_fee"] = 0
    
    #
    df.loc[label_with_fee_fun(df, 100.0), "Target_Label_fee"]   = 1 
    df.loc[label_with_fee_fun(df, 1.0), "Target_2.0_Label_fee"] = 1 
    df.loc[label_with_fee_fun(df, 1.2), "Target_2.2_Label_fee"] = 1 
    df.loc[label_with_fee_fun(df, 1.4), "Target_2.4_Label_fee"] = 1 
    df.loc[label_with_fee_fun(df, 1.6), "Target_2.6_Label_fee"] = 1 
    df.loc[label_with_fee_fun(df, 1.8), "Target_2.8_Label_fee"] = 1 
    df.loc[label_with_fee_fun(df, 2.0), "Target_3.0_Label_fee"] = 1 
    df.loc[label_with_fee_fun(df, 2.2), "Target_3.2_Label_fee"] = 1 
    df.loc[label_with_fee_fun(df, 2.4), "Target_3.4_Label_fee"] = 1 
    df.loc[label_with_fee_fun(df, 2.6), "Target_3.6_Label_fee"] = 1 
    df.loc[label_with_fee_fun(df, 2.8), "Target_3.8_Label_fee"] = 1 
    df.loc[label_with_fee_fun(df, 3.0), "Target_4.0_Label_fee"] = 1 
    #
    df = df[['Date', 'Time', "Low", "Close",'z_open',
       'z_high', 'z_low', 'z_close', 'z_d5', 'z_d10', 'z_d15', 'z_d20',
       'z_d25', 'z_d30', 'rsi', 'macd', 'v_macd', 'natr', 'Target_Label',
             
       'Target_2.0_Label', 'Target_2.2_Label', 'Target_2.4_Label','Target_2.6_Label','Target_2.8_Label','Target_3.0_Label',
       'Target_3.2_Label', 'Target_3.4_Label', 'Target_3.6_Label', 'Target_3.8_Label', 'Target_4.0_Label', 'Target_Label_fee',
      'Target_2.0_Label_fee', 'Target_2.2_Label_fee', 'Target_2.4_Label_fee','Target_2.6_Label_fee','Target_2.8_Label_fee','Target_3.0_Label_fee',
       'Target_3.2_Label_fee', 'Target_3.4_Label_fee', 'Target_3.6_Label_fee', 'Target_3.8_Label_fee', 'Target_4.0_Label_fee', ]]
    
    #print(df.iloc[:, 17:])
    out_df = pd.concat([df.iloc[:, :4], df.iloc[:, 4:18].shift(1), df.iloc[:, 18:]], axis = 1)
    out_df = out_df.iloc[30:, :]
    out_df = out_df.replace(np.nan, 0)
    out_df = out_df.reset_index().iloc[:, 1:]
    
    return out_df# out_df

### Preprocessing Futeres 

In [8]:
def text_to_time_data(txt_name):
    # text data를 data로 출력하는 함수
    # 출력된 함수, shape[0](총행), Time갯수 최대 24

    data = pd.read_csv(txt_name, sep = ",", engine='python', encoding = "cp949", header=None)
    data.columns = ["Date", "Open", "High", "Low","Close","Volume"]
    date_columns = data["Date"].str.split(" ", expand = True)

    data["Date"] = date_columns.iloc[:, 0]
    data["Time"] = date_columns.iloc[:, 1]

    data = data[["Date", "Time", "Open", "High", "Low","Close","Volume"]]
    data = data[(data["Date"] >= "2010-01-04") & (data["Date"] <= "2022-08-01")].reset_index().iloc[:, 1:]
    
    return data, data.shape[0] ,len(data["Time"].unique())
    

In [9]:
# Volumne test
n = []
v_a = []
time = []

for i in range(len( glob.glob("./stock_data/*.txt"))):
    
    name = glob.glob("./stock_data/*.txt")[i]
    if name == "./stock_data/NIY_continuous_adjusted_1hour.txt":
        pass
    if name == "./stock_data/MFS_continuous_adjusted_1hour.txt":
        pass
    else:
        
        a, s,e = text_to_time_data(name)

        n.append(name)                            # G_continuous_adjusted_1hour
        v_a.append(a["Close"].std() * np.sqrt(s)) # 변동성 
        time.append(e)


In [10]:
dataset = pd.DataFrame({"data" : n, "close_vol": v_a, "time" : time})
dataset = dataset[dataset["time"] == 24].sort_values("close_vol", ascending = False)
dataset = list(dataset["data"])

In [11]:
dataset = dataset[:9]

## Make up dataset

In [12]:
def get_dt_col(data):
    col_dt = []
    
    for i in range(len(data)):
        date = str(data['Date'][i]).split(' ')[0]
        time = data['Time'][i]
        dt = date + ' ' + time
        dt = pd.to_datetime(dt)
        col_dt.append(dt)
    return col_dt

In [13]:
def make_up_dataset(dataset):
    for i in range(len(dataset)):
        
        
        data = pd.read_csv(dataset[i], sep = ",", engine='python', encoding = "cp949", header=None) # 여기서 dataset[0] 하나만가져옴
        data_name = dataset[i].split("/")[-1].split("_")[0]
        
        print("Name :",data_name)

        data.columns = ["Date", "Open", "High", "Low","Close","Volume"]
        date_columns = data["Date"].str.split(" ", expand = True)
        data["Date"] = date_columns.iloc[:, 0]
        data["Time"] = date_columns.iloc[:, 1]
        data["Date"] = pd.to_datetime(data["Date"])

        unique_date = pd.DataFrame((data.groupby("Date").count() >= 21))         # 21로 잡으면 모드 칸 데이터 생성 가능
        unique_date = unique_date[unique_date["Open"] == 1].reset_index()["Date"]

        data = data.merge(unique_date).reset_index().iloc[:, 1:]
        data['dt'] = get_dt_col(data)
        dates = data['Date'].drop_duplicates()
        data = data[['dt','Open','High','Low','Close','Volume']].set_index('dt')

        timesteps = [4,8,12,24] #4h, 8h, 12h, 24h

        # timestep = timesteps[0] # for
        # date = dates[0] # for

        data_timestep = {} # 여기 timestep별 변환된 DataFrame 저장

        for timestep in timesteps:

            converted_dict = {} # 해당 timestep의 결과 저장

            for date in dates:

                st = date + timedelta(hours=1) # start time: 01:00:00 부터 시작해야 함

                daily_info = data.loc[st:date+timedelta(days=1)] # 해당일의 데이터 점검

                if (len(daily_info) < 21):
                    # 위에서 21시간 이상 있는거만 필터링 하긴 했지만
                    # 혹시 몰라서 데이터 포인트가 21시간 미만으로 있으면 그 날짜 스킵
                    continue


                for j in range(int(24 / timestep)):
                    et = st + timedelta(hours = timestep-1) # end_time 설정

                    # 해당 timestep 시간 간격의 data
                    step_data = data.loc[st:et]

                    # 해당 timestep에서 새로운 OHLCV 추출
                    step_open = step_data['Open'].iloc[0]
                    step_high = step_data['High'].max()
                    step_low = step_data['Low'].min()
                    step_close = step_data['Close'].iloc[-1]
                    step_volume = step_data['Volume'].sum()

                    # end_time의 시각에 결과물 저장
                    converted_dict[et] = {'Open':step_open,
                                         'High':step_high,
                                         'Low':step_low,
                                         'Close':step_close,
                                         'Volume':step_volume}
                    st = et + timedelta(hours=1)

            data_timestep[timestep] = pd.DataFrame(converted_dict).T



        data_04h = date_split_fun(data_timestep[4].reset_index().rename(columns = {"index" : "Date"}))
        data_08h = date_split_fun(data_timestep[8].reset_index().rename(columns = {"index" : "Date"}))
        data_12h = date_split_fun(data_timestep[12].reset_index().rename(columns = {"index" : "Date"}))
        data_24h = data_timestep[24].reset_index().rename(columns = {"index" : "Date"})
        data_24h["Time"] = "00:00:00"

        a = prepro_stock(data_04h)
        b = prepro_stock(data_08h)
        c = prepro_stock(data_12h)
        d = prepro_stock(data_24h)

        print("Low" == a.columns[2])
        print("Low" == b.columns[2])
        print("Low" == c.columns[2])
        print("Low" == d.columns[2])

        a.to_csv(f"./prepro_2/{data_name}_4h.csv")
        b.to_csv(f"./prepro_2/{data_name}_8h.csv")
        c.to_csv(f"./prepro_2/{data_name}_12h.csv")
        d.to_csv(f"./prepro_2/{data_name}_24h.csv")

In [14]:
make_up_dataset(dataset)

Name : YM
True
True
True
True
Name : NIY
True
True
True
True
Name : NKD
True
True
True
True
Name : NQ
True
True
True
True
Name : ES
True
True
True
True
Name : PA
True
True
True
True
Name : EW
True
True
True
True
Name : RTY
True
True
True
True
Name : PL
True
True
True
True


In [27]:
import zipfile
import os

def zip_folder(folder_path, output_path):
    # make sure folder paths end with a slash
    if not folder_path.endswith('/'):
        folder_path = folder_path + '/'
    if not output_path.endswith('/'):
        output_path = output_path + '/'

    # zip the folder
    zip_file = zipfile.ZipFile(output_path + 'archive.zip', 'w', zipfile.ZIP_DEFLATED)
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            zip_file.write(os.path.join(root, file))
    zip_file.close()

zip_folder('./prepro_2', '/data/Stock_US')

In [2]:
import os
os.getcwd()

'/data/Stock_US'

In [22]:
dataset

['./stock_data/YM_continuous_adjusted_1hour.txt',
 './stock_data/NKD_continuous_adjusted_1hour.txt',
 './stock_data/NQ_continuous_adjusted_1hour.txt',
 './stock_data/ES_continuous_adjusted_1hour.txt',
 './stock_data/PA_continuous_adjusted_1hour.txt',
 './stock_data/EW_continuous_adjusted_1hour.txt',
 './stock_data/RTY_continuous_adjusted_1hour.txt',
 './stock_data/PL_continuous_adjusted_1hour.txt',
 './stock_data/GC_continuous_adjusted_1hour.txt',
 './stock_data/MGC_continuous_adjusted_1hour.txt',
 './stock_data/MFS_continuous_adjusted_1hour.txt',
 './stock_data/MME_continuous_adjusted_1hour.txt',
 './stock_data/VX_continuous_adjusted_1hour.txt',
 './stock_data/CL_continuous_adjusted_1hour.txt',
 './stock_data/UB_continuous_adjusted_1hour.txt',
 './stock_data/BZ_continuous_adjusted_1hour.txt',
 './stock_data/US_continuous_adjusted_1hour.txt',
 './stock_data/HE_continuous_adjusted_1hour.txt',
 './stock_data/ZN_continuous_adjusted_1hour.txt',
 './stock_data/DX_continuous_adjusted_1hour.t

In [19]:
a = pd.read_csv("./prepro_2/NIY_4h.csv")
b = pd.read_csv("./prepro_2/NIY_8h.csv")
c = pd.read_csv("./prepro_2/NIY_12h.csv")
d = pd.read_csv("./prepro_2/NIY_24h.csv")

In [20]:
target_label_list = ['Target_Label', 'Target_2.0_Label', 'Target_2.2_Label',
       'Target_2.4_Label', 'Target_2.6_Label', 'Target_2.8_Label',
       'Target_3.0_Label', 'Target_3.2_Label', 'Target_3.4_Label',
       'Target_3.6_Label', 'Target_3.8_Label', 'Target_4.0_Label']


In [26]:
d[target_label_list].sum()

Target_Label        1223.0
Target_2.0_Label     917.0
Target_2.2_Label     944.0
Target_2.4_Label     971.0
Target_2.6_Label     993.0
Target_2.8_Label    1022.0
Target_3.0_Label    1044.0
Target_3.2_Label    1063.0
Target_3.4_Label    1082.0
Target_3.6_Label    1097.0
Target_3.8_Label    1107.0
Target_4.0_Label    1117.0
dtype: float64

In [21]:
print("4 hour :" ,   (a[target_label_list].sum()[0] - a[target_label_list].sum()[1:]))
print("8 hour :"  ,  (b[target_label_list].sum()[0] - b[target_label_list].sum()[1:]))
print("12 hour :"  , (c[target_label_list].sum()[0] - c[target_label_list].sum()[1:]))
print("24 hour :"  , (d[target_label_list].sum()[0] - d[target_label_list].sum()[1:]))

4 hour : Target_2.0_Label    313.0
Target_2.2_Label    238.0
Target_2.4_Label    184.0
Target_2.6_Label    154.0
Target_2.8_Label    123.0
Target_3.0_Label    109.0
Target_3.2_Label     91.0
Target_3.4_Label     76.0
Target_3.6_Label     65.0
Target_3.8_Label     50.0
Target_4.0_Label     47.0
dtype: float64
8 hour : Target_2.0_Label    398.0
Target_2.2_Label    317.0
Target_2.4_Label    253.0
Target_2.6_Label    203.0
Target_2.8_Label    165.0
Target_3.0_Label    140.0
Target_3.2_Label    121.0
Target_3.4_Label    101.0
Target_3.6_Label     84.0
Target_3.8_Label     72.0
Target_4.0_Label     66.0
dtype: float64
12 hour : Target_2.0_Label    368.0
Target_2.2_Label    301.0
Target_2.4_Label    251.0
Target_2.6_Label    196.0
Target_2.8_Label    162.0
Target_3.0_Label    139.0
Target_3.2_Label    116.0
Target_3.4_Label    102.0
Target_3.6_Label     85.0
Target_3.8_Label     71.0
Target_4.0_Label     70.0
dtype: float64
24 hour : Target_2.0_Label    306.0
Target_2.2_Label    279.0
Target_

In [22]:
print("Stop loss Label (Leverage 5 times) / Origin Label")
print("4 hour :" ,   (a[target_label_list].sum()[1:] / a[target_label_list].sum()[0]).round(3) * 100, "%")
print("8 hour :"  ,  (b[target_label_list].sum()[1:] / b[target_label_list].sum()[0]).round(3) * 100, "%")
print("12 hour :"  , (c[target_label_list].sum()[1:] / c[target_label_list].sum()[0]).round(3) * 100, "%")
print("24 hour :"  , (d[target_label_list].sum()[1:] / d[target_label_list].sum()[0]).round(3) * 100, "%")

Stop loss Label (Leverage 5 times) / Origin Label
4 hour : Target_2.0_Label    95.6
Target_2.2_Label    96.7
Target_2.4_Label    97.4
Target_2.6_Label    97.9
Target_2.8_Label    98.3
Target_3.0_Label    98.5
Target_3.2_Label    98.7
Target_3.4_Label    98.9
Target_3.6_Label    99.1
Target_3.8_Label    99.3
Target_4.0_Label    99.3
dtype: float64 %
8 hour : Target_2.0_Label    89.2
Target_2.2_Label    91.4
Target_2.4_Label    93.1
Target_2.6_Label    94.5
Target_2.8_Label    95.5
Target_3.0_Label    96.2
Target_3.2_Label    96.7
Target_3.4_Label    97.3
Target_3.6_Label    97.7
Target_3.8_Label    98.0
Target_4.0_Label    98.2
dtype: float64 %
12 hour : Target_2.0_Label    84.8
Target_2.2_Label    87.6
Target_2.4_Label    89.6
Target_2.6_Label    91.9
Target_2.8_Label    93.3
Target_3.0_Label    94.3
Target_3.2_Label    95.2
Target_3.4_Label    95.8
Target_3.6_Label    96.5
Target_3.8_Label    97.1
Target_4.0_Label    97.1
dtype: float64 %
24 hour : Target_2.0_Label    75.0
Target_2.2_

In [17]:
print("4 hour :" ,   (a[target_label_list].sum()[0] - a[target_label_list].sum()[1:]))
print("8 hour :"  ,  (b[target_label_list].sum()[0] - b[target_label_list].sum()[1:]))
print("12 hour :"  , (c[target_label_list].sum()[0] - c[target_label_list].sum()[1:]))
print("24 hour :"  , (d[target_label_list].sum()[0] - d[target_label_list].sum()[1:]))

4 hour : Target_2.0_Label    875.0
Target_2.2_Label    739.0
Target_2.4_Label    618.0
Target_2.6_Label    520.0
Target_2.8_Label    452.0
Target_3.0_Label    391.0
Target_3.2_Label    343.0
Target_3.4_Label    289.0
Target_3.6_Label    245.0
Target_3.8_Label    213.0
Target_4.0_Label    184.0
dtype: float64
8 hour : Target_2.0_Label    892.0
Target_2.2_Label    779.0
Target_2.4_Label    696.0
Target_2.6_Label    616.0
Target_2.8_Label    543.0
Target_3.0_Label    481.0
Target_3.2_Label    419.0
Target_3.4_Label    374.0
Target_3.6_Label    327.0
Target_3.8_Label    287.0
Target_4.0_Label    253.0
dtype: float64
12 hour : Target_2.0_Label    773.0
Target_2.2_Label    693.0
Target_2.4_Label    625.0
Target_2.6_Label    547.0
Target_2.8_Label    489.0
Target_3.0_Label    447.0
Target_3.2_Label    396.0
Target_3.4_Label    351.0
Target_3.6_Label    318.0
Target_3.8_Label    280.0
Target_4.0_Label    251.0
dtype: float64
24 hour : Target_2.0_Label    478.0
Target_2.2_Label    455.0
Target_

In [18]:
print("Stop loss Label (Leverage 5 times) / Origin Label")
print("4 hour :" ,   (a[target_label_list].sum()[1:] / a[target_label_list].sum()[0]).round(3) * 100, "%")
print("8 hour :"  ,  (b[target_label_list].sum()[1:] / b[target_label_list].sum()[0]).round(3) * 100, "%")
print("12 hour :"  , (c[target_label_list].sum()[1:] / c[target_label_list].sum()[0]).round(3) * 100, "%")
print("24 hour :"  , (d[target_label_list].sum()[1:] / d[target_label_list].sum()[0]).round(3) * 100, "%")

Stop loss Label (Leverage 5 times) / Origin Label
4 hour : Target_2.0_Label    87.8
Target_2.2_Label    89.7
Target_2.4_Label    91.4
Target_2.6_Label    92.8
Target_2.8_Label    93.7
Target_3.0_Label    94.6
Target_3.2_Label    95.2
Target_3.4_Label    96.0
Target_3.6_Label    96.6
Target_3.8_Label    97.0
Target_4.0_Label    97.4
dtype: float64 %
8 hour : Target_2.0_Label    75.8
Target_2.2_Label    78.8
Target_2.4_Label    81.1
Target_2.6_Label    83.3
Target_2.8_Label    85.3
Target_3.0_Label    86.9
Target_3.2_Label    88.6
Target_3.4_Label    89.8
Target_3.6_Label    91.1
Target_3.8_Label    92.2
Target_4.0_Label    93.1
dtype: float64 %
12 hour : Target_2.0_Label    68.1
Target_2.2_Label    71.4
Target_2.4_Label    74.2
Target_2.6_Label    77.4
Target_2.8_Label    79.8
Target_3.0_Label    81.6
Target_3.2_Label    83.7
Target_3.4_Label    85.5
Target_3.6_Label    86.9
Target_3.8_Label    88.5
Target_4.0_Label    89.6
dtype: float64 %
24 hour : Target_2.0_Label    60.9
Target_2.2_